# TP3: Detector de SPAM

**Integrantes:**
- Carreras, Sebastian
- Fajardo, Lucas
- Más Valdecantos, Marcelo A.

---
Uno de los problemas más comunes en la clasificación es la detección de correos electrónicos SPAM. Uno de los primeros modelos utilizados para abordar este problema fue el clasificador de Bayes ingenuo. La detección de SPAM es un problema persistente en el mundo digital, ya que los spammers continúan adaptando sus estrategias para eludir los filtros de correo no deseado. Además del clasificador de Bayes ingenuo, se han desarrollado y utilizado una variedad de técnicas más avanzadas en la detección de SPAM, que incluyen algoritmos de aprendizaje automático, redes neuronales y métodos basados en reglas.

En este trabajo práctico, utilizaremos un conjunto de datos que consta de 4601 observaciones de correos electrónicos, de los cuales 2788 son correos legítimos y 1813 son correos SPAM. Dado que el contenido de los correos electrónicos es un tipo de dato no estructurado, es necesario procesarlo de alguna manera. Para este conjunto de datos, ya se ha aplicado un procesamiento típico en el Procesamiento del Lenguaje Natural (NLP), que consiste en contar la frecuencia de palabras observadas en los correos.

El procesamiento de lenguaje natural (NLP) desempeña un papel fundamental en la detección de SPAM, ya que permite analizar el contenido de los correos electrónicos y extraer características relevantes para la clasificación. Además de contar la frecuencia de palabras, se pueden utilizar técnicas más sofisticadas, como la extracción de características semánticas y el análisis de sentimientos, para mejorar la precisión de los modelos de detección de SPAM.

En este proceso, se cuenta la cantidad de ocurrencias de cada palabra en los diferentes correos.

![spam counter](./spam.png)

Con el fin de preservar la privacidad de los mensajes, la frecuencia de palabras se encuentra normalizada. El conjunto de datos está compuesto por 54 columnas de atributos que se denominan:

- `word_freq_XXXX`: Donde `XXXX` es la palabra o símbolo. Los valores son enteros que van de 0 a 20k.

Además, hay una columna adicional llamada `spam`, que es 1 si el correo es SPAM o 0 si no lo es.

Los clasificadores de Bayes ingenuos fueron los primeros filtros utilizados por las aplicaciones de correo electrónico, basados en este principio de palabras. La idea es que, partiendo de un dato a priori sobre la probabilidad de que un correo sea SPAM o no, ciertas palabras nos indicarán que la probabilidad a posteriori, dadas esas palabras, es más probable que el correo sea SPAM o no.

---
#### 1.¿Cuáles son las 10 palabras más encontradas en correos con SPAM y en correos No SPAM? ¿Hay palabras en común? ¿Algunas llaman la atención?


In [145]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Scikit-learn nos ofrece una variedad ampliada de modelos Naive Bayes, para este problema usamos MultinomialNB que es pensado para este tipo de problemas
from sklearn.naive_bayes import MultinomialNB   

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay

In [147]:
dataset = pd.read_csv("dataset/spambase.csv") # cargando los datos desde un CSV
dataset.head(100)
dataset.rename(columns={"word_freq_make\r": "word_freq_make"}, inplace=True)

In [149]:
#Obtenemos los palabras mas usadas. 
column_sum = dataset.groupby(by="spam", as_index=False).sum()

In [151]:
#  Filtramos los datos para solo los correos etiquetados como spam
spam_word_counts = column_sum[column_sum['spam'] == 1]

In [153]:
# Eliminamos la columna Spam 
spam_word_counts = spam_word_counts.drop(columns=["spam"])

In [155]:
# Transponemos el DataFrame para que las palabras sean los índices
transposed_spam_word_counts = spam_word_counts.T

# Renombramos la columna resultante, en caso de que no tenga un nombre adecuado
transposed_spam_word_counts.columns = ['numero_de_palabras']

# Ordenamos las palabras por su frecuencia en orden descendente
most_frequent_words = transposed_spam_word_counts.sort_values(by='numero_de_palabras', ascending=False)

# Mostramos las n palabras más frecuentes
n = 10
print(most_frequent_words.head(n))


                    numero_de_palabras
word_freq_you                  4105599
word_freq_your                 2502597
word_freq_will                  997100
word_freq_free                  939790
word_freq_our                   931799
char_freq_!                     931352
word_freq_all                   732080
word_freq_mail                  635470
word_freq_email                 578759
word_freq_business              521250


In [157]:
# Hacemos lo mismo para las mails que no son spam
no_spam_word_counts = column_sum[column_sum['spam'] == 0]

In [159]:
# Eliminamos la columna Spam 
no_spam_word_counts = no_spam_word_counts.drop(columns=["spam"])

In [161]:
# Transponemos el DataFrame para que las palabras sean los índices
transposed_no_spam_word_counts = no_spam_word_counts.T

# Renombramos la columna resultante, en caso de que no tenga un nombre adecuado
transposed_no_spam_word_counts.columns = ['numero_de_palabras']

# Ordenamos las palabras por su frecuencia en orden descendente
no_spam_most_frequent_words = transposed_no_spam_word_counts.sort_values(by='numero_de_palabras', ascending=False)

# Mostramos las n palabras más frecuentes en mails que no son Spam 
n = 10
print(no_spam_most_frequent_words.head(n))

                   numero_de_palabras
word_freq_you                 3541702
word_freq_george              3527559
word_freq_hp                  2496576
word_freq_will                1495268
word_freq_your                1223098
word_freq_hpl                 1204398
word_freq_re                  1159138
word_freq_edu                  800669
word_freq_address              681569
word_freq_meeting              604460


---

#### Respuesta 1
##### ¿Hay palabras en común?
- Podemos Observar que la palabra "you" es la que más se repite en el caso de Spam y No Spam, las otras palabras que más se repiten en ambos casos son "will" y "your".
##### ¿Algunas llaman la atención?
- La palabra que llama la atención, o más bien el signo, por su frecuencia de aparición en el correo de spam, es el signo de admiración “!”.



---
#### 2. Separe el conjunto de datos en un conjunto de entrenamiento y un conjunto de prueba (70% y 30% respectivamente).

In [165]:
# Obtenemos los atributos y target
X = (dataset.drop(columns="spam") * 100).astype(int)
#X = dataset2.drop(columns="spam")
y = dataset["spam"]

In [167]:
# Separamos el conjunto de datos como fue indicado.
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size = 0.3)

---
#### 3. Utilizando un clasificador de Bayes ingenuo, entrene con el conjunto de entrenamiento.

In [170]:
#Escalamos los datos
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Lo transformamos en DataFrames
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns)

In [172]:
X_train_scaled

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,word_freq_re,word_freq_edu,word_freq_table,word_freq_conference,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#
0,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.0000,0.000000,0.0,0.0,0.000000,0.121206,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.0000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.0,0.000,0.668605,0.000000,0.000000,0.00000,0.000000,...,0.0000,0.000000,0.0,0.0,0.000000,0.089007,0.000000,0.151273,0.144594,0.000000
3,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.0000,0.000000,0.0,0.0,0.000000,0.037326,0.044597,0.000000,0.000000,0.000000
4,0.000000,0.018207,0.182028,0.0,0.026,0.075581,0.011004,0.229437,0.10076,0.210596,...,0.0130,0.003628,0.0,0.0,0.023406,0.023175,0.000000,0.018452,0.042312,0.005699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3215,0.257709,0.246499,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.00000,0.154967,...,0.0000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.021640,0.000000,0.000000
3216,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.541126,0.00000,0.000000,...,0.0625,0.000000,0.0,0.0,0.000000,0.019278,0.000000,0.000000,0.031318,0.000000
3217,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.0000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3218,0.262115,0.000000,0.135945,0.0,0.000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.0000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.005332,0.000000,0.000000


In [181]:
#Entrenamos el modelo de bayes ingenuo multinomial
naives_bayes_spam = MultinomialNB()
naives_bayes_spam.fit(X_train_scaled, y_train)

MultinomialNB()

---
#### 4. Utilizando un clasificador de Regresión Logística, entrene con el conjunto de entrenamiento (en este caso, normalice los datos).

In [209]:
classifier_balance_spam = LogisticRegression(random_state = 0, class_weight="balanced")

In [211]:
classifier_balance_spam.fit(X_train_scaled, y_train)

LogisticRegression(class_weight='balanced', random_state=0)

---
#### 5. Calcule la matriz de confusión del conjunto de evaluación para ambos modelos. ¿Qué tipo de error comete más cada modelo? ¿Cuál de los dos tipos de error crees que es más importante para este problema?

In [222]:
# Hacer predicciones con ambos modelos
y_pred_logistic = classifier_balance_spam.predict(X_test_scaled)
y_pred_naive_bayes = naives_bayes_spam.predict(X_test_scaled)

In [226]:
# Calcular las matrices de confusión de LR
y_pred_logistic_binary = np.where(y_pred_logistic > 0.5, 1, 0)

In [228]:
# Calcular las matrices de confusión de NB
confusion_logistic = confusion_matrix(y_test, y_pred_logistic_binary)  # Ensure y_test is defined

In [230]:
print("Matriz de Confusión - Regresión Logística:")
print(confusion_logistic)

Matriz de Confusión - Regresión Logística:
[[746  91]
 [ 67 477]]


In [238]:
# Calcular las matrices de confusión de LR
y_pred_naive_bayes_binary = np.where(y_pred_naive_bayes > 0.5, 1, 0)

In [240]:
confusion_naive_bayes = confusion_matrix(y_test, y_pred_naive_bayes_binary)  

In [242]:
print("Matriz de Confusión - Regresión Logística:")
print(confusion_naive_bayes)

Matriz de Confusión - Regresión Logística:
[[786  51]
 [116 428]]
